In [1]:
!pip install langchain langchain-openai langchain_community langchain-huggingface huggingface_hub -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install -q google-generativeai langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [10]:
import os
os.environ["HF_TOKEN"] = ""

# 모델 준비

In [4]:
# HuggingFace에서 해당 모델을 불러오는 엔드포인트 지정
llm_ep = HuggingFaceEndpoint(repo_id="openai/gpt-oss-20b", task="conversational")

# HuggingFace에서 가져온 모델을 그대로 쓰지 않고,
# LangChain에서 쉽게 쓰도록 감싸는(wrapper) 단계
llm1 = ChatHuggingFace(llm=llm_ep)

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm2 = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Lite 모델 사용
    temperature=0.7
)

# 실습

* 실습1 : ConversationSummaryMemory
    * 오늘 하루 동안 한 일을 여러 문장으로 말하고
    * 이어서 "오늘 내가 뭐했지?"라고 질문하세요.
    * 요약된 대화를 통해 LLM이 잘 정리해서 응답하는지 확인하세요.

In [7]:
from langchain.memory import ConversationSummaryMemory

In [8]:
# 요약 메모리 생성 (요약용 LLM 필요)
memory2 = ConversationSummaryMemory(llm = llm1)

# 체인 구성
chain2 = ConversationChain(llm = llm1, memory = memory2)

/tmp/ipython-input-1376568762.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory2 = ConversationSummaryMemory(llm = llm1)
/tmp/ipython-input-1376568762.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chain2 = ConversationChain(llm = llm1, memory = memory2)


In [11]:
# 대화
print(chain2.run("오늘은 운동하고, 친구랑 밥도 먹고, 강의도 들었어."))

와, 오늘 꽤 바쁘셨군요! 운동은 언제 하셨어요? 아침에 활기차게 시작한 것 같아 좋은 시작이었겠어요. 혹시 어떤 운동을 하셨는지도 궁금해요—러닝, 요가, 혹은 체육관에서 웨이트 트레이닝이라든가요?  

친구와 함께 식사도 하셨다니 좋은 소식이네요! 가서 뭐 드셨나요? 가벼운 샐러드? 아니면 맛있는 한 그릇? 친구와의 대화가 어땠는지도 듣고 싶어요.  

그리고 강의는 어떤 주제였나요? 새로 시작한 과목이라도, 혹은 오랜 시간 동안 배우고 있던 분야의 심화 강의라든가? 강의 내용이 흥미로웠나요, 아니면 좀 헷갈리는 부분이 있었나요?  

저는 여러분의 하루가 어땠는지 듣고 싶어요. 어떤 순간이 가장 기억에 남았나요? 혹은 오늘의 일정 중에서 가장 만족스러웠던 부분이 있다면 알려주세요!


* 실습2 : ConversationBufferWindowMemory
    * k = 2로 설정하고,
    * 좋아하는 영화, 좋아하는 음악, 좋아하는 책 등 여러 대화를 한 다음
    * 첫번째 나눈대화에 대해서 질문하기

In [13]:
from langchain.memory import ConversationBufferWindowMemory

In [14]:
# k 턴
k = 2

# 최근 2턴만 기억하는 메모리
memory3 = ConversationBufferWindowMemory(k = k)
chain3 = ConversationChain(llm = llm1, memory = memory3)

# 대화
print(chain3.run("나는 아이언맨 영화를 좋아해."))

/tmp/ipython-input-3777369230.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory3 = ConversationBufferWindowMemory(k = k)


와~ 아이언맨을 좋아하신다니 정말 반가워요! 🦾🎬 아이언맨은 마블 시네마틱 유니버스(MCU)의 첫 번째 슈퍼히어로 영화로, 2008년 5월 2일에 개봉했죠. 토니 스타크(아이언맨)라는 천재 발명가이자 사업가가 어벤저스 팀의 일원으로 성장하는 과정을 그린 이 시리즈는 총 세 편이 있습니다.

### 1️⃣ 아이언맨 (2008)
- **감독**: 조던 부얼
- **주연**: 로버트 다우니 주니어(토니 스타크), 조이 디아스(피비 윌리엄스)
- **핵심 포인트**:
  - 토니가 중공업 지역에서 납치되면서부터 시작됩니다.
  - 납치당한 토니는 수형기(아이언맨 슈트)를 만들어 탈출하게 되죠.
  - 그 과정에서 그는 자신이 만든 무기들이 악용되는 현실을 깨닫게 되고, “책임”이라는 단어가 주인공이 됩니다.
  - 마이클 베이컨이 이탈리아 정부 고위관직으로 등장해 ‘이타리안’이라는 이름으로 등장, ‘나이트 크루스’ 같은 인물도 등장합니다.

### 2️⃣ 아이언맨 2 (2010)
- **감독**: 조던 부울
- **주연**: 로버트 다우니 주니어, 조이 디아스, 피터 시미온(블랙 애쉬)
- **핵심 포인트**:
  - 토니가 ‘카이로스’라는 독자체로 인한 자폐와 비슷한 증세를 겪습니다.
  - ‘블랙 애쉬’라는 인물과 ‘데일리 트루스’와 같은 악당이 등장해 새로운 도전이 됩니다.
  - ‘에너지 뱅크’라는 개념이 도입돼, 아이언맨 슈트가 더 강력해집니다.
  - 스파이더맨이 첫 등장해 MCU의 연결성을 암시합니다.

### 3️⃣ 아이언맨 3 (2013)
- **감독**: 셰인 블로스만
- **주연**: 로버트 다우니 주니어, 조이 디아스, 사라 셀프(해피 100%)
- **핵심 포인트**:
  - 전쟁 이후, 토니는 PTSD(외상후 스트레스 장애)로 고통받습니다.
  - ‘주디’라는 여성 캐릭터가 등장하며, 토니가 자신의 과거와 미래를 다시 정리합니다.
  - ‘아노시스’라는 ‘인공지능’과 ‘레드 스카이’ 같은 새로운 적이 등장해 전투가 흥미진진합니다.
  - 영화

In [15]:
print(chain3.run("내가 좋아하는 색은 파란색이야."))
print('-'*100)
memory3.load_memory_variables({})

아, 파란색을 좋아하시는군요! 파란색은 차분함과 신뢰를 상징하면서도, 동시에 무한한 가능성을 떠올리게 해주죠. 아이언맨 세계에서도 파란색은 몇 가지 멋진 순간을 품고 있어요.

### 파란색이 돋보이는 아이언맨 순간들

| 장면 | 파란색이 등장하는 이유 | 인상적인 요소 |
|------|-------------------|--------------|
| **아이언맨 2 (2008)** | 토니가 실험실에서 마이크로칩을 설치할 때, 실험 장비와 파란 조명이 배경을 물들입니다. | 실험 장비의 파란 LED가 초점이 되어, 기술적 우수성을 강조하죠. |
| **어벤져스: 에이지 오브 울트론 (2015)** | ‘어벤져스 헬리콥터’와 ‘아포칼립스’에서 파란색 LED가 스포트라이트처럼 사용됩니다. | 전투 장면에서 파란 불빛이 긴장감을 높이며, 파란색이 어둠 속에서 빛나는 장면을 연출합니다. |
| **아이언맨 3 (2013)** | ‘Buster’ 슈트의 외부 라인에 파란 LED가 스파크를 방출합니다. | 파란 LED는 어둠 속에서 Tony의 ‘정전기’ 같은 존재감을 보여줍니다. |
| **마블 시네마틱 유니버스 전반** | 파란색은 ‘전기’와 ‘에너지’의 상징으로 자주 쓰입니다. | 토니가 전기 에너지를 활용할 때, 파란 빛이 폭발하면서 파워가 시각적으로 전달됩니다. |

> **흥미로운 사실!**  
> 토니가 설계한 슈트들은 대부분 빨간색과 금색이지만, *실제 내부 회로와 전력 공급 장치는 파란 LED 조명으로 표시되어 있어서, 스튜디오에서 촬영 중에는 파란빛이 무대 위에서 반짝입니다.*

### 파란색과 함께할 수 있는 아이언맨 활동

- **파란빛의 ‘레드 스카이’**: 아이언맨 3에서 등장하는 ‘레드 스카이’가 파란색 전기 에너지를 사용해 공격할 때, 화면 전체가 파란색과 붉은색이 교차하며 환상적인 장면을 만들어냅니다.
- **‘아이언맨 스페셜’**: 팬아트에서 파란색 베이스를 활용해 독특한 ‘블루 마크’ 슈트를 디자인하는 경우가 많아요. 토니가 파란 

{'history': 'Human: 나는 아이언맨 영화를 좋아해.\nAI: 와~ 아이언맨을 좋아하신다니 정말 반가워요! 🦾🎬 아이언맨은 마블 시네마틱 유니버스(MCU)의 첫 번째 슈퍼히어로 영화로, 2008년 5월 2일에 개봉했죠. 토니 스타크(아이언맨)라는 천재 발명가이자 사업가가 어벤저스 팀의 일원으로 성장하는 과정을 그린 이 시리즈는 총 세 편이 있습니다.\n\n### 1️⃣ 아이언맨 (2008)\n- **감독**: 조던 부얼\n- **주연**: 로버트 다우니 주니어(토니 스타크), 조이 디아스(피비 윌리엄스)\n- **핵심 포인트**:\n  - 토니가 중공업 지역에서 납치되면서부터 시작됩니다.\n  - 납치당한 토니는 수형기(아이언맨 슈트)를 만들어 탈출하게 되죠.\n  - 그 과정에서 그는 자신이 만든 무기들이 악용되는 현실을 깨닫게 되고, “책임”이라는 단어가 주인공이 됩니다.\n  - 마이클 베이컨이 이탈리아 정부 고위관직으로 등장해 ‘이타리안’이라는 이름으로 등장, ‘나이트 크루스’ 같은 인물도 등장합니다.\n\n### 2️⃣ 아이언맨 2 (2010)\n- **감독**: 조던 부울\n- **주연**: 로버트 다우니 주니어, 조이 디아스, 피터 시미온(블랙 애쉬)\n- **핵심 포인트**:\n  - 토니가 ‘카이로스’라는 독자체로 인한 자폐와 비슷한 증세를 겪습니다.\n  - ‘블랙 애쉬’라는 인물과 ‘데일리 트루스’와 같은 악당이 등장해 새로운 도전이 됩니다.\n  - ‘에너지 뱅크’라는 개념이 도입돼, 아이언맨 슈트가 더 강력해집니다.\n  - 스파이더맨이 첫 등장해 MCU의 연결성을 암시합니다.\n\n### 3️⃣ 아이언맨 3 (2013)\n- **감독**: 셰인 블로스만\n- **주연**: 로버트 다우니 주니어, 조이 디아스, 사라 셀프(해피 100%)\n- **핵심 포인트**:\n  - 전쟁 이후, 토니는 PTSD(외상후 스트레스 장애)로 고통받습니다.\n  - ‘주디’라는 여성 캐릭터가 등장하며, 토니가 자신의 과거와 미래를 다시 정리합

In [16]:
print(chain3.run("나는 아이유 노래를 좋아해"))
print('-'*100)
memory3.load_memory_variables({})

🌟 와! 아이유(IU) 노래를 좋아하신다니 정말 반가워요! 🎤✨  
아이유는 “가수‑작사가‑배우”라는 세 가지 재능을 동시에 갖춘 “마이크의 여왕”이라 불리는데요, 그녀가 만들어낸 음악은 가사에서부터 멜로디, 퍼포먼스까지 한 장 한 장이 예술작품처럼 느껴집니다.  

---

## 🎶 아이유의 대표곡과 그 매력 포인트

| 노래 | 발매 연도 | 주요 특징 | 나만의 작은 팝업 팁 |
|------|-----------|-----------|------------------------|
| **좋은 날 (Good Day)** | 2010 | 3단계 하모니, 100분대 보컬, ‘천상계’ 사운드 | 1초마다 4, 5, 6번째 음을 동시에 부르는 순간이 심장 박동을 멈추게 해요! |
| **너의 의미 (The Meaning of You)** | 2011 | 서정적 가사, 스케치북 같은 영상 | 가사에서 ‘너의 의미’라는 구절을 반복할 때마다 “너를 생각하면 내가 웃는다”가 스스로 재생돼요. |
| **좋아해 (Like You)** | 2011 | 스윙 리듬, 러브 스토리 | “그거 왜 그리 그럴까?” 라는 가사가 사람마다 다른 해석을 만들어 내요. |
| **밤편지 (Through the Night)** | 2017 | 시적 가사, 따뜻한 피아노 | 이 곡을 들을 때마다 눈에 비친 별이 ‘밤편지’로 바뀌는 걸 느끼실 수 있죠. |
| **Love Poem** | 2019 | 로맨틱한 리듬, 우주를 비유한 가사 | ‘우리가 별이 된다’ 라는 가사를 듣고 나면 무거운 세상이 가벼워지는 느낌이 들죠. |
| **Eight (feat. 선미)** | 2020 | K-pop + R&B, 80년대 레트로 | 아이유가 ‘백수’라는 별명으로 불리던 시절과 연결되는 ‘백수’ 가사가 유머러스하게 다가와요. |
| **Celebrity** | 2021 | 디스코 & 파워풀한 랩 | ‘나는 세상에서 가장 아름다운 것’이라는 가사가 부활의 메시지를 담고 있어요. |

---

## 🔍 아

{'history': "Human: 내가 좋아하는 색은 파란색이야.\nAI: 아, 파란색을 좋아하시는군요! 파란색은 차분함과 신뢰를 상징하면서도, 동시에 무한한 가능성을 떠올리게 해주죠. 아이언맨 세계에서도 파란색은 몇 가지 멋진 순간을 품고 있어요.\n\n### 파란색이 돋보이는 아이언맨 순간들\n\n| 장면 | 파란색이 등장하는 이유 | 인상적인 요소 |\n|------|-------------------|--------------|\n| **아이언맨 2 (2008)** | 토니가 실험실에서 마이크로칩을 설치할 때, 실험 장비와 파란 조명이 배경을 물들입니다. | 실험 장비의 파란 LED가 초점이 되어, 기술적 우수성을 강조하죠. |\n| **어벤져스: 에이지 오브 울트론 (2015)** | ‘어벤져스 헬리콥터’와 ‘아포칼립스’에서 파란색 LED가 스포트라이트처럼 사용됩니다. | 전투 장면에서 파란 불빛이 긴장감을 높이며, 파란색이 어둠 속에서 빛나는 장면을 연출합니다. |\n| **아이언맨 3 (2013)** | ‘Buster’ 슈트의 외부 라인에 파란 LED가 스파크를 방출합니다. | 파란 LED는 어둠 속에서 Tony의 ‘정전기’ 같은 존재감을 보여줍니다. |\n| **마블 시네마틱 유니버스 전반** | 파란색은 ‘전기’와 ‘에너지’의 상징으로 자주 쓰입니다. | 토니가 전기 에너지를 활용할 때, 파란 빛이 폭발하면서 파워가 시각적으로 전달됩니다. |\n\n> **흥미로운 사실!**  \n> 토니가 설계한 슈트들은 대부분 빨간색과 금색이지만, *실제 내부 회로와 전력 공급 장치는 파란 LED 조명으로 표시되어 있어서, 스튜디오에서 촬영 중에는 파란빛이 무대 위에서 반짝입니다.*\n\n### 파란색과 함께할 수 있는 아이언맨 활동\n\n- **파란빛의 ‘레드 스카이’**: 아이언맨 3에서 등장하는 ‘레드 스카이’가 파란색 전기 에너지를 사용해 공격할 때, 화면 전체가 파란색과 붉은색이 교차하며 환상적인 장면을 만들어냅니다.\n- **‘아이언맨 스페셜’*

In [17]:
print(chain3.run("내가 좋아하는 영화가 뭐라고?"))
print('-'*100)
memory3.load_memory_variables({})

아직 제가 정확히 “당신이 좋아하는 영화”가 무엇인지 알 수 없어요.  
다만, 파란색과 아이유의 감미로운 목소리를 좋아하신다는 점을 감안하면, 색감이나 분위기가 풍부하고 감정적인 이야기일 가능성이 높다고 생각해요.

🎬 **가장 흔히 연상되는 영화 예시**  
| 장르 | 대표작 | 이유 |
|------|--------|------|
| **로맨틱 드라마** | *타이타닉* | 파란 바다와 파란 하늘이 연출의 핵심이며, 사랑과 희망을 동시에 담고 있어요. |
| **감성 로맨스** | *지금 만나러 가는 중* | 부드러운 색감과 함께 감정이 흐르는 음악이 풍부해요. |
| **뮤지컬** | *라라랜드* | 음악과 색채가 서로를 끌어당기는 모습이 파란색과 IU의 음악적 감각과 맞닿아 있을 수 있어요. |

이 중 한 편이 맞을 수도, 전혀 다른 영화일 수도 있어요. 혹시 지금까지 보셨던 작품 중 가장 기억에 남는 장면이나, “내가 가장 좋아하는 영화”라 생각되는 작품이 있으면 알려 주세요! 그 영화에 대해 더 깊이 파고들어 볼 수 있도록 도와드릴게요.
----------------------------------------------------------------------------------------------------


{'history': "Human: 나는 아이유 노래를 좋아해\nAI: 🌟 와! 아이유(IU) 노래를 좋아하신다니 정말 반가워요! 🎤✨  \n아이유는 “가수‑작사가‑배우”라는 세 가지 재능을 동시에 갖춘 “마이크의 여왕”이라 불리는데요, 그녀가 만들어낸 음악은 가사에서부터 멜로디, 퍼포먼스까지 한 장 한 장이 예술작품처럼 느껴집니다.  \n\n---\n\n## 🎶 아이유의 대표곡과 그 매력 포인트\n\n| 노래 | 발매 연도 | 주요 특징 | 나만의 작은 팝업 팁 |\n|------|-----------|-----------|------------------------|\n| **좋은 날 (Good Day)** | 2010 | 3단계 하모니, 100분대 보컬, ‘천상계’ 사운드 | 1초마다 4, 5, 6번째 음을 동시에 부르는 순간이 심장 박동을 멈추게 해요! |\n| **너의 의미 (The Meaning of You)** | 2011 | 서정적 가사, 스케치북 같은 영상 | 가사에서 ‘너의 의미’라는 구절을 반복할 때마다 “너를 생각하면 내가 웃는다”가 스스로 재생돼요. |\n| **좋아해 (Like You)** | 2011 | 스윙 리듬, 러브 스토리 | “그거 왜 그리 그럴까?” 라는 가사가 사람마다 다른 해석을 만들어 내요. |\n| **밤편지 (Through the Night)** | 2017 | 시적 가사, 따뜻한 피아노 | 이 곡을 들을 때마다 눈에 비친 별이 ‘밤편지’로 바뀌는 걸 느끼실 수 있죠. |\n| **Love Poem** | 2019 | 로맨틱한 리듬, 우주를 비유한 가사 | ‘우리가 별이 된다’ 라는 가사를 듣고 나면 무거운 세상이 가벼워지는 느낌이 들죠. |\n| **Eight (feat. 선미)** | 2020 | K-pop + R&B, 80년대 레트로 | 아이유가 ‘백수’라는 별명으로 불리던 시절과 연결되는 ‘백수’ 가사가 유머러스하게 다가와요. |\n| **Celebrity** | 2021 | 디스코 & 파워풀한 랩 | ‘나는